In [2]:
import os
import pandas as pd
from datetime import datetime
import unicodedata

In [3]:
brands = pd.read_json('brands.json', lines=True)
brands['_id'] = brands['_id'].apply(lambda x: list(x.values())[0])

brands = brands.rename(columns={'_id':'brandId'})
brands['barcode'] = brands['barcode'].astype(str)
#brands.head()

In [4]:
cpg = brands[['brandId','cpg']].copy()
cpg['oid'] = cpg['cpg'].apply(lambda x: list(x.values())[0] if len(list(x.values())[0]) != 4 else list(x.values())[1])
cpg['oid'] = cpg['oid'].apply(lambda x: list(x.values())[0])
cpg['ref'] = cpg['cpg'].apply(lambda x: list(x.values())[1] if len(list(x.values())[1]) == 4 else list(x.values())[0])
cpg = cpg[['brandId','oid','ref']]
brands = pd.merge(brands, cpg, how = 'left', on='brandId')

In [5]:
brands.head()

,brandId,barcode,category,categoryCode,cpg,name,topBrand,brandCode,oid,ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


In [6]:
def date(x):
    if type(x) == float:
        pass
    else:
        return(pd.to_datetime(list(x.values())[0], unit='ms'))

def date1(x):
    if type(x) == float:
        pass
    else:
        return(x)

In [7]:
receipts = pd.read_json('receipts.json', lines=True)
receipts['_id'] = receipts['_id'].apply(lambda x: list(x.values())[0])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date(x))
receipts['modifyDate'] = receipts['finishedDate'].apply(lambda x: date1(x))
receipts['pointsAwardedDate'] = receipts['finishedDate'].apply(lambda x: date1(x))
receipts['purchaseDate'] = receipts['finishedDate'].apply(lambda x: date1(x))

receipts = receipts.rename(columns={'_id':'receiptId'})

In [8]:
def explode(df, col):
    df[col] = df[col].apply(lambda x: [x] if not isinstance(x, list) else x)
    return df.drop(col, axis=1).join(pd.DataFrame(list(df[col])).stack().reset_index(level=1, drop=True).rename(col)) 

def explode1(x, key):
    if type(x) == dict and key in x.keys(): return x[key]

In [9]:
Itemslist = receipts[['rewardsReceiptItemList','receiptId','userId']].copy()
Itemslist = explode(Itemslist,"rewardsReceiptItemList").reset_index()
Itemslist = Itemslist.rename(columns={'index':'group'})

In [10]:
keys_ = []
for i in Itemslist['rewardsReceiptItemList']:
    if type(i) != float:
        for j in i.keys():
            if j not in keys_:
                keys_.append(j)


In [11]:
Itemslist_ = pd.DataFrame(columns=keys_)

for i in range(len(keys_)):
    this_column = Itemslist_.columns[i]
    Itemslist_[this_column] = Itemslist['rewardsReceiptItemList'].apply(lambda x: explode1(x,this_column))

Itemslist = pd.concat([Itemslist,Itemslist_],axis=1)
# Itemslist = Itemslist.rename(columns = {'barcode': 'productId'})

In [12]:
users = pd.read_json('users.json', lines=True)
users['_id'] = users['_id'].apply(lambda x: list(x.values())[0])
users['createdDate'] = users['createdDate'].apply(lambda x: date(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date(x))

users = users.rename(columns = {'_id':'userId'}) 

In [13]:
receipts.drop(['rewardsReceiptItemList'],axis=1, inplace = True)
brands.drop(['cpg'],axis = 1,inplace = True)
Itemslist.drop(['rewardsReceiptItemList'],axis=1,inplace = True)

In [14]:
print(users.columns)
len(users) 

Index(['userId', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')


495

In [15]:
print(Itemslist.columns)

Index(['group', 'receiptId', 'userId', 'barcode', 'description', 'finalPrice',
       'itemPrice', 'needsFetchReview', 'partnerItemId',
       'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode',
       'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity',
       'needsFetchReviewReason', 'pointsNotAwardedReason', 'pointsPayerId',
       'rewardsGroup', 'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')


### I saved the tables into csv files.

In [16]:
brands.to_csv('brands.csv',encoding='utf-8')
Itemslist.to_csv('Itemslist.csv',encoding='utf-8')
users.to_csv('users.csv',encoding='utf-8')
receipts.to_csv('receipts.csv',encoding='utf-8')

## Part3: Data Issues

### We first check for duplicate rows

In [17]:

print('Duplicate rows in brands:\n',
      brands.duplicated().value_counts(),'\n') 
print('Duplicate rows in receipts:\n',
      receipts.duplicated().value_counts(),'\n') 

print('Duplicate rows in users:\n',
      users.duplicated().value_counts(),'\n') 

print('duplicate rows in Itemslist:\n',
      Itemslist.duplicated().value_counts(),'\n')

Duplicate rows in brands:
 False    1167
dtype: int64 

Duplicate rows in receipts:
 False    1119
dtype: int64 

Duplicate rows in users:
 True     283
False    212
dtype: int64 

duplicate rows in Itemslist:
 False    7381
dtype: int64 



##### we have 283 duplicate rows in users table

### Then we check for missing values

In [18]:
print ('brands dataset ({} rows) \n null value of each variable:\n'.format(brands.shape[0]),
       '\n',brands.isnull().sum(),'\n')
print ('receipts dataset ({} rows) \n null value of each variable:\n'.format(receipts.shape[0]),
       '\n',receipts.isnull().sum(),'\n')
print ('users dataset ({} rows) \n null value of each variable:\n'.format(users.shape[0]),
       '\n',users.isnull().sum(),'\n')
print ('Itemslist dataset ({} rows) \n null value of each variable:\n'.format(Itemslist.shape[0]),
       '\n',Itemslist.isnull().sum(),'\n')

brands dataset (1167 rows) 
 null value of each variable:
 
 brandId           0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
oid               0
ref               0
dtype: int64 

receipts dataset (1119 rows) 
 null value of each variable:
 
 receiptId                    0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                 551
pointsAwardedDate          551
pointsEarned               510
purchaseDate               551
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64 

users dataset (495 rows) 
 null value of each variable:
 
 userId           0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64 

Itemslist dataset (7381 r

#### this is understandable because when we unfold the json files there are a lot of rows with null values.

## We then explore some problems with the barcode

In [19]:
Itemslist['barcode'].unique

<bound method Series.unique of 0               4011
1               4011
2       028400642255
3               None
4               4011
            ...     
7376            None
7377            None
7378      B076FJ92M4
7379      B07BRRLSVC
7380            None
Name: barcode, Length: 7381, dtype: object>

In [20]:
brands['barcode'].unique

<bound method Series.unique of 0       511111019862
1       511111519928
2       511111819905
3       511111519874
4       511111319917
            ...     
1162    511111116752
1163    511111706328
1164    511111416173
1165    511111400608
1166    511111019930
Name: barcode, Length: 1167, dtype: object>

#### As we can see, brands has 1167 barcodes and itemlist has 7381, which is not matched.

In [22]:
missing = round(100*Itemslist['barcode'].isnull().sum()/len(Itemslist),2)
print('There are %s%s items in receipts item list that does not have a barcode.' % (missing,'%'))

There are 58.14% items in receipts item list that does not have a barcode.


In [22]:
merged_barcode = pd.merge(left = Itemslist, right = brands,how = 'inner', on = ['barcode'])
percentage = round(100*len(merged_barcode)/len(Itemslist['barcode'].unique()),2)
print('Only %s%s items in receipts item list can be found in brands data set.' % (percentage,'%'))

Only 15.64% items in receipts item list can be found in brands data set.


#### We then check for problems in the user data set

In [23]:
users_issue = []
for i in receipts['userId'].unique():
    if i not in users['userId'].unique():
        users_issue.append(i)
print('%d users by receipts scanned are not found in users data set.' %len(users_issue))

117 users by receipts scanned are not found in users data set.
